# Unitary Devices Examples

1. Two-Port Coupler Coherent States
2. Three-Port Coupler with Two Coherent States and One Vaccum State
3. Three-Port Jx Coupler with Three Input Photons

## 1. Two-Port Coupler with Two Coherent States

In [ ]:
# Library imports

import numpy as np
import uuid
from qsi.coordinator import Coordinator
from qsi.state import State, StateProp
from IPython.display import display

In [ ]:
# Waveguide coupler properties
N_ports   = 2 # Number of waveguides
L           = np.pi/4 # meters
n = 3.45

# Coherent states properties
truncation  = 15
alpha_1     = 1.0+0j # Coherence parameter - Intensity is proportional to alpha**2
alpha_2     = 2.1+0j # Coherence parameter - Intensity is proportional to alpha**2
wavelength = 1550.0
bandwidth = 10.0
polarization = "H" # Horizontal or TE

In [ ]:
# Create coordinator and start the modules:
coordinator = Coordinator(port=23456)
# Register coherent source module for two photons
cs1 = coordinator.register_component(module="coherent_source.py", runtime="python")
cs2 = coordinator.register_component(module="coherent_source.py", runtime="python")
# Register multiport coupler
coupler = coordinator.register_component(module="jx_coupler.py", runtime="python")
# Run the coordinator process
coordinator.run()

In [ ]:
# Configure the module parameters
# Set up coherent source 1
cs1.set_param("alpha", alpha_1)
cs1.send_params()

# Set up coherent source 2
cs2.set_param("alpha", alpha_2)
cs2.send_params()

# Set up coupler
coupler.set_param("n_ports", N_ports)
coupler.set_param("length", L)
coupler.set_param("n", n)
coupler.send_params()

In [ ]:
# Generate the space for coherent states
cs1_state_prop = StateProp(
    state_type="light",
    truncation=truncation,
    wavelength=wavelength,
    bandwidth=bandwidth,
    polarization=polarization,
)
cs1_state = State(cs1_state_prop)

cs2_state_prop = StateProp(
    state_type="light",
    truncation=truncation,
    wavelength=wavelength,
    bandwidth=bandwidth,
    polarization=polarization,
)
cs2_state = State(cs2_state_prop)

In [ ]:
%%capture

# Suppress output due to the large truncation term

# Get Kraus operators for the generation of the coherent state 1
response1, operators1 = cs1.channel_query(
    cs1_state, {"input": cs1_state_prop.uuid}
)

# Apply Kraus operators to the state 1
cs1_state.apply_kraus_operators(
    operators1,
    cs1_state.get_all_props(response1["kraus_state_indices"])
)

In [ ]:
%%capture

# Get Kraus operators for the generation of the coherent state 2
response2, operators2 = cs2.channel_query(
    cs2_state, {"input": cs2_state_prop.uuid}
)

# Apply Kraus operators to the state 2
cs2_state.apply_kraus_operators(
    operators2,
    cs2_state.get_all_props(response2["kraus_state_indices"])
)

In [ ]:
# Join the states
cs1_state.join(cs2_state)

# Copy the input state
input_state = cs1_state.state.copy()

In [ ]:
%%capture

# Inject photons into 2-port Jx coupler
response3, operators3 = coupler.channel_query(
    cs1_state, {"input": [cs1_state_prop.uuid, cs2_state_prop.uuid]}
)

# Apply kraus operators
cs1_state.apply_kraus_operators(
    operators3,
    cs1_state.get_all_props(response3["kraus_state_indices"])
)

In [ ]:
# Terminate the coordinator process
coordinator.terminate()

## 2. Three-Port Coupler with Two Coherent States and One Vacuum State

In [ ]:
# Library imports

import numpy as np
import uuid
from qsi.coordinator import Coordinator
from qsi.state import State, StateProp
from IPython.display import display

In [ ]:
# Waveguide coupler properties
N_ports   = 3 # Number of waveguides
L           = np.pi/4 # meters
n = 3.45

# Photon states properties
alpha_1     = 1.0+0j # Coherence parameter - Intensity is proportional to alpha**2
alpha_2     = 2.5+0j # Coherence parameter - Intensity is proportional to alpha**2
truncation  = 5
wavelength = 1550.0
bandwidth = 10.0
polarization = "H" # Horizontal or TE

In [ ]:
# Create coordinator and start the modules:
coordinator = Coordinator(port=23457)
# Register module for photon states for each input channel
cs1 = coordinator.register_component(module="coherent_source.py", runtime="python")
cs2 = coordinator.register_component(module="coherent_source.py", runtime="python")
ps1 = coordinator.register_component(module="multi_photon_source.py", runtime="python")
# Register multiport coupler
coupler = coordinator.register_component(module="jx_coupler.py", runtime="python")
# Run the coordinator process
coordinator.run()

In [ ]:
# Configure the module parameters

# Set up coherent source 1
cs1.set_param("alpha", alpha_1)
cs1.send_params()

# Set up coherent source 2
cs2.set_param("alpha", alpha_2)
cs2.send_params()

# Set up vacuum state
ps1.set_param("n_photons", 0)
ps1.send_params()

# Set up coupler
coupler.set_param("n_ports", N_ports)
coupler.set_param("length", L)
coupler.set_param("n", n)
coupler.send_params()

In [ ]:
# Generate the space for coherent states
cs1_state_prop = StateProp(
    state_type="light",
    truncation=truncation,
    wavelength=wavelength,
    bandwidth=bandwidth,
    polarization=polarization,
)
cs1_state = State(cs1_state_prop)

cs2_state_prop = StateProp(
    state_type="light",
    truncation=truncation,
    wavelength=wavelength,
    bandwidth=bandwidth,
    polarization=polarization,
)
cs2_state = State(cs2_state_prop)

# Generate the space for the vacuum state
ps1_state_prop = StateProp(
    state_type="light",
    truncation=truncation,
    wavelength=wavelength,
    bandwidth=bandwidth,
    polarization=polarization,
)
ps1_state = State(ps1_state_prop)

In [ ]:
%%capture

# Suppress output due to the large truncation term

# Get Kraus operators for the generation of the coherent state 1
response1, operators1 = cs1.channel_query(
    cs1_state, {"input": cs1_state_prop.uuid}
)

# Apply Kraus operators to the state 1
cs1_state.apply_kraus_operators(
    operators1,
    cs1_state.get_all_props(response1["kraus_state_indices"])
)

# Get Kraus operators for the generation of the coherent state 2
response2, operators2 = cs2.channel_query(
    cs2_state, {"input": cs2_state_prop.uuid}
)

# Apply Kraus operators to the state 2
cs2_state.apply_kraus_operators(
    operators2,
    cs2_state.get_all_props(response2["kraus_state_indices"])
)

# Get Kraus operators for the generation of the photon state 1
response3, operators3 = ps1.channel_query(
    ps1_state, {"input": ps1_state_prop.uuid}
)

# Apply Kraus operators to the photon state 3
ps1_state.apply_kraus_operators(
    operators3,
    ps1_state.get_all_props(response3["kraus_state_indices"])
)

In [ ]:
# DEBUG
cs1_state_copy = cs1_state.state.copy()

# Join the states
cs1_state.join(cs2_state)
cs1_state.join(ps1_state)

# Copy the input state
input_state = cs1_state.state.copy()

In [ ]:
%%capture

# Inject states into 3-port Jx coupler
response4, operators4 = coupler.channel_query(
    cs1_state, {"input": [cs1_state_prop.uuid, cs2_state_prop.uuid, ps1_state_prop.uuid]}
)

# Apply kraus operators
cs1_state.apply_kraus_operators(
    operators4,
    cs1_state.get_all_props(response4["kraus_state_indices"])
)

In [ ]:
# Terminate the coordinator process
coordinator.terminate()

## 3. Three-Port Jx Coupler with Three Input Photons

Input state:
- 0 photons (vacuum state) in port 1
- 1 photon in port 2
- 2 photons in port 3

In [ ]:
# Library imports

import numpy as np
import uuid
from qsi.coordinator import Coordinator
from qsi.state import State, StateProp
from IPython.display import display

In [ ]:
# Waveguide coupler properties
N_ports   = 3 # Number of waveguides
L           = np.pi/4 # meters
n = 3.45

# Photon states properties
N_list = [0, 1, 2]
truncation  = 5
wavelength = 1550.0
bandwidth = 10.0
polarization = "H" # Horizontal or TE

# Ensure the array size matches the number of ports
assert(len(N_list) == N_ports)

In [ ]:
# Create coordinator and start the modules:
coordinator = Coordinator(port=23458)
# Register module for photon states for each input channel
ps1 = coordinator.register_component(module="multi_photon_source.py", runtime="python")
ps2 = coordinator.register_component(module="multi_photon_source.py", runtime="python")
ps3 = coordinator.register_component(module="multi_photon_source.py", runtime="python")
# Register multiport coupler
coupler = coordinator.register_component(module="jx_coupler.py", runtime="python")
# Run the coordinator process
coordinator.run()

In [ ]:
# Configure the module parameters
# Set up photon states
ps1.set_param("n_photons", N_list[0])
ps1.send_params()

ps2.set_param("n_photons", N_list[1])
ps2.send_params()

ps3.set_param("n_photons", N_list[2])
ps3.send_params()

# Set up coupler
coupler.set_param("n_ports", N_ports)
coupler.set_param("length", L)
coupler.set_param("n", n)
coupler.send_params()

In [ ]:
# Generate the space for photons states
ps1_state_prop = StateProp(
    state_type="light",
    truncation=truncation,
    wavelength=wavelength,
    bandwidth=bandwidth,
    polarization=polarization,
)
ps1_state = State(ps1_state_prop)

ps2_state_prop = StateProp(
    state_type="light",
    truncation=truncation,
    wavelength=wavelength,
    bandwidth=bandwidth,
    polarization=polarization,
)
ps2_state = State(ps2_state_prop)

ps3_state_prop = StateProp(
    state_type="light",
    truncation=truncation,
    wavelength=wavelength,
    bandwidth=bandwidth,
    polarization=polarization,
)
ps3_state = State(ps3_state_prop)

In [ ]:
%%capture

# Suppress output due to the large truncation term

# Get Kraus operators for the generation of the photon state 1
response1, operators1 = ps1.channel_query(
    ps1_state, {"input": ps1_state_prop.uuid}
)

# Apply Kraus operators to the photon state 1
ps1_state.apply_kraus_operators(
    operators1,
    ps1_state.get_all_props(response1["kraus_state_indices"])
)

# Get Kraus operators for the generation of the photon state 1
response2, operators2 = ps2.channel_query(
    ps2_state, {"input": ps2_state_prop.uuid}
)

# Apply Kraus operators to the photon state 2
ps2_state.apply_kraus_operators(
    operators2,
    ps2_state.get_all_props(response2["kraus_state_indices"])
)

# Get Kraus operators for the generation of the photon state 3
response3, operators3 = ps3.channel_query(
    ps3_state, {"input": ps3_state_prop.uuid}
)

# Apply Kraus operators to the photon state 3
ps3_state.apply_kraus_operators(
    operators3,
    ps3_state.get_all_props(response3["kraus_state_indices"])
)

In [ ]:
# Join the states
ps1_state.join(ps2_state)
ps1_state.join(ps3_state)

# Copy the input state
input_state = ps1_state.state.copy()

In [ ]:
%%capture

# Inject photons into 3-port Jx coupler
response4, operators4 = coupler.channel_query(
    ps1_state, {"input": [ps1_state_prop.uuid, ps2_state_prop.uuid, ps3_state_prop.uuid]}
)

# Apply kraus operators
ps1_state.apply_kraus_operators(
    operators4,
    ps1_state.get_all_props(response4["kraus_state_indices"])
)

In [ ]:
# Terminate the coordinator process
coordinator.terminate()